In [3]:
import re
import dateparser

In [21]:
# TODO:
# - test for different date patterns and update patterns if needed
# - weiterbildung


In [6]:
date_patterns = [
    re.compile(r'^\s*\b(\d{4})\b\s*[-–]?\s*(heute|JETZT|Jetzt|jetzt)', re.IGNORECASE),
    re.compile(r'\b(\d{4})\b\s*[-–]?\s*(bis\s*)?(heute|JETZT|Jetzt|jetzt)', re.IGNORECASE),
    re.compile(r'\b(\d{4})\s*[-–]\s*(\d{4})\b'),
    re.compile(r'^\s*[\w\-.]*\s*(\d{4})\s*[-–]\s*(\d{4})\b'),  
    re.compile(r'\b(\d{2}/\d{4})\s*[-–]\s*(\d{2}/\d{4})|\b(\d{2}/\d{4})\s*[-–]?\s*(heute|JETZT|Jetzt|jetzt)', re.IGNORECASE),
    re.compile(r'^\s*seit\s*(\d{2}/\d{4})\s*bis\s*(dato|JETZT|Jetzt|jetzt|heute)', re.IGNORECASE),
    re.compile(r'^\s*(\b(?:Jan(?:uar)?|Feb(?:ruar)?|Mär(?:z)?|Apr(?:il)?|Mai|Jun(?:i)?|Jul(?:i)?|Aug(?:ust)?|Sep(?:tember)?|Okt(?:ober)|Nov(?:ember)|Dez(?:ember)?)\.?\s(\d{4}))\s*[-–]?\s*(heute|JETZT|Jetzt|jetzt|\b(?:Jan(?:uar)?|Feb(?:ruar)?|Mär(?:z)?|Apr(?:il)?|Mai|Jun(?:i)?|Jul(?:i)?|Aug(?:ust)?|Sep(?:tember)?|Okt(?:ober)|Nov(?:ember)|Dez(?:ember)?\.?\s(\d{4}))?)', re.IGNORECASE),
    re.compile(r'(?:(?:(?:\b\w{3,9}\s\d{4}\b)|(?:\b\d{1,2}/\d{4}\b)|(?:\b\d{4}\b))(?:\s*[-–]?\s*(?:(?:\b\w{3,9}\s\d{4}\b)|(?:\b\d{1,2}/\d{4}\b)|(?:\b\d{4}\b)))?)')
]


In [7]:
def normalize_date(date_text, pattern):
    date_text = date_text.lower().strip()
    precision = 'unknown'

    # "Jahr – bis heute" (e.g., "2020 - heute")
    if pattern == date_patterns[0]:
        year_match = re.search(r'\b\d{4}\b', date_text)
        if year_match:
            year = year_match.group()
            precision = 'year'
            return {"date": f"{year} - Heute", "precision": precision}

    # "Jahr – bis heute" mit "bis" (e.g., "2020 - bis heute")
    elif pattern == date_patterns[1]:
        year_match = re.search(r'\b\d{4}\b', date_text)
        if year_match:
            year = year_match.group()
            precision = 'year'
            return {"date": f"{year} - Heute", "precision": precision}

    # "Jahr zu Jahr" (e.g., "2004 - 2008")
    elif pattern in [date_patterns[2], date_patterns[3]]:
        years = re.findall(r'\d{4}', date_text)
        if len(years) == 2:
            precision = 'year'
            return {"date": f"{years[0]} - {years[1]}", "precision": precision}

    # "Monat/Jahr – Monat/Jahr" (e.g., "03/2018 – 07/2018")
    elif pattern == date_patterns[4]:
        try:
            start, end = date_text.split("–")
            start_obj = dateparser.parse(start.strip())
            end_obj = dateparser.parse(end.strip()) if "heute" not in end.strip().lower() else None
            
            if start_obj:
                start_formatted = start_obj.strftime('%m.%Y') if start_obj.day == 1 else start_obj.strftime('%d.%m.%Y')
                precision = 'month' if start_obj.day == 1 else 'day'
            else:
                return {"date": "Invalid date format", "precision": precision}

            if end_obj:
                end_formatted = end_obj.strftime('%m.%Y') if end_obj.day == 1 else end_obj.strftime('%d.%m.%Y')
            else:
                end_formatted = "Heute"
                
            return {"date": f"{start_formatted} - {end_formatted}", "precision": precision}

        except (ValueError, AttributeError):
            return {"date": "Invalid date format", "precision": precision}

    # TODO: add for more formats ..

    return {"date": date_text, "precision": precision}

In [8]:
def check_date_position(line, normalize = True):
    date_found = False
    date_position = 'none'
    date_text = ""

    for pattern in date_patterns:
        date_match = pattern.search(line)
        if date_match:
            date_text = date_match.group(0).strip()
            date_found = True
        
            if normalize:
                date_text = normalize_date(date_text, pattern)["date"]
            
            stripped_line = line.strip()

            if stripped_line.startswith(date_text):
                date_position = 'start'
            elif stripped_line.endswith(date_text):
                date_position = 'end'
            else:
                date_position = 'middle'

            break

    return {
        "date_found": date_found,
        "position": date_position,
        "date_text": date_text
    }


In [9]:
def handle_start_date(current_point, date, description, structured_points):
    if current_point:
        structured_points.append(current_point)

    current_point = {"date": date, "description": description.strip()}
    return current_point

In [10]:
def handle_end_date(current_point, date, description, structured_points):
    if current_point:
        current_point["description"] += " " + description.strip()
        structured_points.append(current_point)

    current_point = {"date": date, "description": description.strip()}
    return current_point

In [11]:
def handle_middle_date(current_point, description):
    # TODO
    if current_point:
        current_point["description"] += " " + description
    return current_point

In [ ]:
def get_education(text):
    education_points = text.splitlines()
    structured_points = []
    current_point = None

    for line in education_points:
        # TODO: doesn't work with normalize here
        result = check_date_position(line, False)

        if result['date_found']:
            date = result['date_text']
            description = line.replace(date, '').strip()

            if result['position'] == 'start':
                current_point = handle_start_date(current_point, date, description, structured_points)

            elif result['position'] == 'end':
                current_point = handle_end_date(current_point, date, description, structured_points)

            elif result['position'] == 'middle':
                current_point = handle_middle_date(current_point, description)

        else:
            if current_point:
                current_point["description"] += " " + line.strip()

    if current_point:
        structured_points.append(current_point)

    return structured_points

In [ ]:
# example with date at beginning of lines
education_section = """
10/2020 – Bachelor of Science in Informatik
Fokus auf Softwareentwicklung und Datenbanken.
Abschlussarbeit über Künstliche Intelligenz.

03/2021 – Praktikum bei der Firma Beispiel AG
Hier habe ich an mehreren Projekten zur Webentwicklung gearbeitet.
Das Praktikum umfasste sowohl Frontend- als auch Backend-Entwicklung.

09/2022 – Master of Science in Data Science
Studium an der Universität Beispielstadt.
Vertiefung in maschinellem Lernen und Datenanalyse.

11/2023 – Weiterbildung in Projektmanagement
Verbesserung meiner Fähigkeiten in der Teamleitung.
Erstellung effektiver Projektpläne und -strategien.

seit 2019 – Freiwilliges Engagement beim Tierschutzverein
Aktive Mitwirkung bei der Organisation von Veranstaltungen.
Unterstützung von Tierschutzprojekten in der Region.
"""

parsed_education = get_education(education_section)

for point in parsed_education:
    print(point)


{'date': '10/2020', 'description': '– Bachelor of Science in Informatik Fokus auf Softwareentwicklung und Datenbanken. Abschlussarbeit über Künstliche Intelligenz. '}
{'date': '03/2021', 'description': '– Praktikum bei der Firma Beispiel AG Hier habe ich an mehreren Projekten zur Webentwicklung gearbeitet. Das Praktikum umfasste sowohl Frontend- als auch Backend-Entwicklung. '}
{'date': '09/2022', 'description': '– Master of Science in Data Science Studium an der Universität Beispielstadt. Vertiefung in maschinellem Lernen und Datenanalyse. '}
{'date': '11/2023', 'description': '– Weiterbildung in Projektmanagement Verbesserung meiner Fähigkeiten in der Teamleitung. Erstellung effektiver Projektpläne und -strategien. '}
{'date': 'seit 2019', 'description': '– Freiwilliges Engagement beim Tierschutzverein Aktive Mitwirkung bei der Organisation von Veranstaltungen. Unterstützung von Tierschutzprojekten in der Region.'}


In [14]:
# example with date at end of lines
education_section = """
Bachelor of Science in Informatik – 10/2020
Fokus auf Softwareentwicklung und Datenbanken.
Abschlussarbeit über Künstliche Intelligenz.

Praktikum bei der Firma Beispiel AG – 03/2021
Hier habe ich an mehreren Projekten zur Webentwicklung gearbeitet.
Das Praktikum umfasste sowohl Frontend- als auch Backend-Entwicklung.

Master of Science in Data Science – 09/2022
Studium an der Universität Beispielstadt.
Vertiefung in maschinellem Lernen und Datenanalyse.

Weiterbildung in Projektmanagement – 11/2023
Verbesserung meiner Fähigkeiten in der Teamleitung.
Erstellung effektiver Projektpläne und -strategien.

Freiwilliges Engagement beim Tierschutzverein – bis heute
Aktive Mitwirkung bei der Organisation von Veranstaltungen.
Unterstützung von Tierschutzprojekten in der Region.
"""

parsed_education = get_education(education_section)

for point in parsed_education:
    print(point)


{'date': '10/2020', 'description': 'Bachelor of Science in Informatik – Fokus auf Softwareentwicklung und Datenbanken. Abschlussarbeit über Künstliche Intelligenz.  Praktikum bei der Firma Beispiel AG –'}
{'date': '03/2021', 'description': 'Praktikum bei der Firma Beispiel AG – Hier habe ich an mehreren Projekten zur Webentwicklung gearbeitet. Das Praktikum umfasste sowohl Frontend- als auch Backend-Entwicklung.  Master of Science in Data Science –'}
{'date': '09/2022', 'description': 'Master of Science in Data Science – Studium an der Universität Beispielstadt. Vertiefung in maschinellem Lernen und Datenanalyse.  Weiterbildung in Projektmanagement –'}
{'date': '11/2023', 'description': 'Weiterbildung in Projektmanagement – Verbesserung meiner Fähigkeiten in der Teamleitung. Erstellung effektiver Projektpläne und -strategien.  Freiwilliges Engagement beim Tierschutzverein – bis heute Aktive Mitwirkung bei der Organisation von Veranstaltungen. Unterstützung von Tierschutzprojekten in de